In [1]:
import torch
from torch.nn import Linear
import actuallysparse.converter as converter
import actuallysparse.layers as layers
import pretrained
import copy

In [2]:
training_device = "cuda" if torch.cuda.is_available() else "cpu"
pruning_device = "cpu"

In [3]:
# Ładowanie modeli
pretrained_model = torch.load(".weights/full/pretrained", map_location=torch.device('cpu'))
extra_trained = torch.load(".weights/full/extra_trained", map_location=torch.device('cpu'))
pruned_model = torch.load(".weights/full/pruned", map_location=torch.device('cpu'))
very_pruned_model = torch.load(".weights/full/very_pruned", map_location=torch.device('cpu'))

In [4]:
dataloader_train, dataloader_test = pretrained.load_cifar10_dataloaders()

Files already downloaded and verified
Files already downloaded and verified


In [5]:
def eval_accuracy(model, dataset="train"):
    dataloader = dataloader_train if dataset == "train" else dataloader_test
    with torch.no_grad():
        model.to(training_device)
        correct = 0
        all_so_far = 0
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(training_device), labels.to(training_device)
            pred = torch.argmax(model(inputs), dim=1)

            all_so_far += labels.size().numel()
            correct += torch.sum(pred.eq(labels))
    return correct/all_so_far

In [6]:
def convolutional_pass(model):
    passed_data = []
    for data in dataloader_train:
        with torch.no_grad():
            inputs, labels = data
            inputs_for_sparse = model.features(inputs)
            inputs_for_sparse = model.avgpool(inputs_for_sparse)
            inputs_for_sparse = inputs_for_sparse.view(inputs_for_sparse.size(0), -1)
            *conv_data, = inputs_for_sparse, labels
            passed_data.append(conv_data)
    return passed_data


In [7]:
def train_prune_loop(model, data, optimizer, criterion,max_epochs = 2000, epochs_to_prune = 15):
    in_classifier_features = model.classifier[0].in_features
    dummy_input = torch.ones(in_classifier_features)
    model.train()
    for epoch in range(max_epochs):
        if epoch % epochs_to_prune == 0:
            layers.prune_model(model.classifier, dummy_input)
            print("Pruned!")
        for entry in data:
            inputs_for_sparse, labels = entry
            optimizer.zero_grad()
            outputs = model.classifier(inputs_for_sparse)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        print(f"Epoch:{epoch}, loss:{loss}")

In [8]:
pruned_model_coo = copy.deepcopy(pruned_model)
pruned_model_coo.classifier = converter.convert_model(pruned_model_coo.classifier, Linear, 'coo')
for child in pruned_model_coo.classifier.children():
    if type(child) is layers.SparseLayer:
        child.set_k(0.03)

In [9]:
optimizer = torch.optim.Adam(pruned_model_coo.classifier.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

In [10]:
pruned_model_coo.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inplace=True)
    (14): MaxPool2d(ke

In [11]:
eval_accuracy(pruned_model_coo, dataset="test")

tensor(0.8365)

In [12]:
passed_data = convolutional_pass(pruned_model_coo)

In [13]:
%%time
train_prune_loop(pruned_model_coo, passed_data, optimizer, criterion, max_epochs=6, epochs_to_prune=2)

Pruned!
Epoch:0, loss:0.07555679976940155
Epoch:1, loss:0.07355543971061707
Pruned!
Epoch:2, loss:0.539714515209198
Epoch:3, loss:0.4877779483795166
Pruned!
Epoch:4, loss:2.0283749103546143
Epoch:5, loss:2.059316396713257
CPU times: total: 11h 51min 25s
Wall time: 3h 29min 27s


In [14]:
pruned_model_coo.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inplace=True)
    (14): MaxPool2d(ke

In [15]:
eval_accuracy(pruned_model_coo, dataset="test")

tensor(0.7480)

In [17]:
very_pruned_model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inplace=True)
    (14): MaxPool2d(ke

In [19]:
eval_accuracy(very_pruned_model, dataset="test")

tensor(0.7657)